In [1]:
import os
import utils.cv_reviewer_utils as cv_utils
from IPython.display import Markdown, display

In [2]:
#if its published somewhere
#job_description = get_job_description("url")

job_description = """

Job Title: Junior Data Engineer
Location: Remote
Job Type: Full-time
Team: Data Engineering
Reports to: Senior Data Engineer

About the Role:

We are looking for a Junior Data Engineer to join our growing data team. You will support the design, development, and maintenance of data pipelines and infrastructure that power data analytics, reporting, and data-driven decision-making across the company.
This is a great opportunity for someone early in their career to learn and grow while working on real-world data challenges in a collaborative and agile environment.

Key Responsibilities
- Assist in building, maintaining, and optimizing ETL/ELT pipelines.
- Work closely with data analysts, scientists, and engineers to support data ingestion and transformation workflows.
- Help ensure the reliability and quality of data across systems.
- Monitor and troubleshoot data pipelines and processes.
- Contribute to documentation of data flows, models, and architecture.
- Learn and apply best practices in data engineering, including security and scalability.

Requirements:

- Basic knowledge of SQL and at least one programming language (Python preferred).
- Familiarity with data storage systems (e.g., relational databases, cloud storage).
- Understanding of data processing concepts and tools.
- Willingness to learn and grow in a fast-paced environment.
- Good communication and problem-solving skills.
- Bachelor’s degree in Computer Science, Engineering, Mathematics, or related field (or equivalent practical experience).

Nice to Have:

- Exposure to cloud platforms like AWS, GCP, or Azure.
- Experience with version control (e.g., Git).
- Basic knowledge of data modeling concepts.
- Familiarity with tools like Airflow, dbt, or Spark.

"""

# in lower case
mandatory_keywrods = ["spark","python"]



In [3]:
landing_path = './landing'

descriptions_dict = {}
evaluation_dict = {}
matches = []

for filename in os.listdir(landing_path):
    path = os.path.join(landing_path, filename)
    
    if os.path.isfile(path):
        cv_text = cv_utils.extract_text_from_cv(path)
        if cv_text is not None:
            
            words = cv_text.split()    
            num_of_words = len(words)

            if num_of_words > 5:    
                descriptions_dict[filename] = cv_text

for filename,candidate_desc in descriptions_dict.items():

        # "llama3.2" or "gpt-4o-mini" by now :)
        model = "gpt-4o-mini"

        # keywords string is not mandatory
        keywords_string = "Additional note: " + cv_utils.evaluate_mandatory_keywords(candidate_desc,mandatory_keywrods)
    
        llm_answer = cv_utils.evaluate_candidate(model, candidate_desc, job_description, keywords_string)
        matches.append(llm_answer)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using gpt-4o-mini via OpenAI API...
▶️ Using gpt-4o-mini via OpenAI API...
▶️ Using gpt-4o-mini via OpenAI API...
▶️ Using gpt-4o-mini via OpenAI API...


In [6]:
sorted_matches = sorted(matches, key=lambda x: x['match_percentage'], reverse=True)

markdown_text = """
### 📊 Candidate Evaluations

"""

n = 1
for match in sorted_matches:
    markdown_text += f"""**🧑‍💼 Candidate nº{n}: {match['name']} – {match['match_percentage']}%**

    {match['summary']}
    
"""
    n+=1

display(Markdown(markdown_text))


### 📊 Candidate Evaluations

**🧑‍💼 Candidate nº1: ALAN SUSA – 90%**

    > Alan has extensive experience as a Data Engineer, demonstrating strong skills in Python and SQL, as well as familiarity with cloud platforms like AWS. He has built and maintained ETL pipelines and has a solid understanding of data processing concepts. However, his experience level exceeds the junior position, which may lead to a mismatch in expectations regarding seniority.
    
**🧑‍💼 Candidate nº2: John Smith – 85%**

    > John has over 5 years of experience as a Data Engineer, demonstrating strong skills in SQL, Python, and ETL processes. He has worked with large datasets and developed scalable databases, which aligns well with the responsibilities of the Junior Data Engineer role. However, his experience may be considered too advanced for a junior position, and there is no mention of familiarity with cloud platforms or version control systems, which are nice to have for this role.
    
**🧑‍💼 Candidate nº3: Thaddeus Drake – 85%**

    > Thaddeus has relevant experience as a Junior Data Engineer, with strong skills in SQL and Python. He has worked with AWS and has experience in building and optimizing ETL pipelines using Apache NiFi and Airflow. He demonstrates familiarity with data storage systems and has a solid educational background in Computer Science. However, while he has exposure to version control with Git, there is no mention of experience with data modeling concepts or Spark.
    
**🧑‍💼 Candidate nº4: Elara Quinn – 75%**

    > Elara has relevant experience as a Data Engineer Intern, with skills in SQL, Apache NiFi, and AWS, which align well with the job requirements. She has built data pipelines and worked with data ingestion and transformation workflows. However, she lacks explicit experience with Python and version control systems like Git, which are preferred for the role.
    
